In [ ]:
import json
from pathlib import Path

# Paths to your GeoJSON files
METRO_GEOJSON = Path("/home/muhammadrehman/Documents/SNA/Lahore_Metro_Route/graph-ml/data/citylines/lahore_sections.geojson")
SPEEDO_GEOJSON = Path("/home/muhammadrehman/Documents/SNA/Lahore_Metro_Route/graph-ml/data/speedo/speedo_sections.geojson")
OUTPUT_GEOJSON = Path("../data/lahore_merged_routes.geojson")

def merge_geojson(files, output_file):
    """Merge multiple GeoJSON files into a single FeatureCollection."""
    merged_features = []

    for file in files:
        print(f"Loading {file}...")
        with open(file, "r", encoding="utf-8") as f:
            data = json.load(f)
            if data.get("type") == "FeatureCollection":
                merged_features.extend(data.get("features", []))
            else:
                print(f"  {file} is not a FeatureCollection, skipping.")

    merged_collection = {
        "type": "FeatureCollection",
        "features": merged_features
    }

    # Write merged GeoJSON
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(merged_collection, f, indent=2, ensure_ascii=False)

    print(f"\nMerged GeoJSON saved to {output_file}")
    print(f"Total features: {len(merged_features)}")

# Merge Metro + Speedo
merge_geojson([METRO_GEOJSON, SPEEDO_GEOJSON], OUTPUT_GEOJSON)


Loading /home/muhammadrehman/Documents/SNA/Lahore_Metro_Route/graph-ml/data/citylines/lahore_sections.geojson...
Loading /home/muhammadrehman/Documents/SNA/Lahore_Metro_Route/graph-ml/data/speedo/speedo_sections.geojson...

Merged GeoJSON saved to ../data/lahore_merged_routes.geojson
Total features: 41


In [2]:
import geopandas as gpd
df = gpd.read_file("../data/lahore_merged_routes.geojson")
print(df.head(5))

      id    klass  length       osm_id  \
0  27066  Section   11900  818457073.0   
1  27067  Section     707  818457074.0   
2  27068  Section     259  818457075.0   
3  27069  Section    1311  818457076.0   
4  27070  Section     200  818457077.0   

                                            osm_tags   osm_metadata  opening  \
0  {"bridge":"yes","electrified":"rail","gauge":"...  {"version":9}     2020   
1  {"bridge":"yes","electrified":"rail","gauge":"...  {"version":6}     2020   
2  {"covered":"yes","cutting":"yes","electrified"...  {"version":6}     2020   
3  {"electrified":"rail","gauge":"1435","layer":"...  {"version":6}     2020   
4  {"cutting":"yes","electrified":"rail","gauge":...  {"version":6}     2020   

                                               lines  \
0  [ { "line": "Orange Line", "line_url_name": "4...   
1  [ { "line": "Orange Line", "line_url_name": "4...   
2  [ { "line": "Orange Line", "line_url_name": "4...   
3  [ { "line": "Orange Line", "line_url_na

In [3]:
df.dropna()

,id,klass,length,osm_id,osm_tags,osm_metadata,opening,lines,geometry
0,27066,Section,11900,818457073.0,"{""bridge"":""yes"",""electrified"":""rail"",""gauge"":""...","{""version"":9}",2020,"[ { ""line"": ""Orange Line"", ""line_url_name"": ""4...","LINESTRING (74.3038 31.55267, 74.3034 31.55248..."
1,27067,Section,707,818457074.0,"{""bridge"":""yes"",""electrified"":""rail"",""gauge"":""...","{""version"":6}",2020,"[ { ""line"": ""Orange Line"", ""line_url_name"": ""4...","LINESTRING (74.30658 31.55841, 74.30654 31.558..."
2,27068,Section,259,818457075.0,"{""covered"":""yes"",""cutting"":""yes"",""electrified""...","{""version"":6}",2020,"[ { ""line"": ""Orange Line"", ""line_url_name"": ""4...","LINESTRING (74.30763 31.56056, 74.30735 31.560..."
3,27069,Section,1311,818457076.0,"{""electrified"":""rail"",""gauge"":""1435"",""layer"":""...","{""version"":6}",2020,"[ { ""line"": ""Orange Line"", ""line_url_name"": ""4...","LINESTRING (74.31878 31.56632, 74.31825 31.566..."
4,27070,Section,200,818457077.0,"{""cutting"":""yes"",""electrified"":""rail"",""gauge"":...","{""version"":6}",2020,"[ { ""line"": ""Orange Line"", ""line_url_name"": ""4...","LINESTRING (74.32087 31.56652, 74.31878 31.56632)"
5,27071,Section,11020,818457078.0,"{""bridge"":""yes"",""electrified"":""rail"",""gauge"":""...","{""version"":11}",2020,"[ { ""line"": ""Orange Line"", ""line_url_name"": ""4...","LINESTRING (74.432 31.58978, 74.4313 31.58972,..."


In [1]:
import networkx as nx
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
from scipy.spatial import cKDTree
import numpy as np

# ------------------------------
# File paths
# ------------------------------
ROAD_GRAPHML = "../data/osmnx/network.graphml"
TRANSIT_GEOJSON = "../data/lahore_merged_routes.geojson"
POPULATION_CSV = "/home/muhammadrehman/Documents/SNA/Lahore_Metro_Route/graph-ml/data/worldpop/pak_pd_2020_1km_UNadj_ASCII_XYZ.csv"  # has X,Y,Z
POIS_GPKG = "/home/muhammadrehman/Documents/SNA/Lahore_Metro_Route/graph-ml/data/osmnx/all-pois.gpkg"  # actual POIs

OUTPUT_GRAPHML = "../data/lahore_full_graph.graphml"

# ------------------------------
# 1. Load road network
# ------------------------------
G = nx.read_graphml(ROAD_GRAPHML)
print(f"Loaded road graph with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")

# Convert node positions to a dict for easier spatial joins
node_positions = {n: (float(G.nodes[n]['x']), float(G.nodes[n]['y'])) for n in G.nodes}

# ------------------------------
# 2. Load population and assign to nearest node
# ------------------------------
pop_df = pd.read_csv(POPULATION_CSV)
# Z is population density
pop_gdf = gpd.GeoDataFrame(pop_df, geometry=gpd.points_from_xy(pop_df.X, pop_df.Y), crs="EPSG:4326")

node_coords = np.array(list(node_positions.values()))
node_ids = list(node_positions.keys())
tree = cKDTree(node_coords)

for idx, row in pop_gdf.iterrows():
    lon, lat = row.geometry.x, row.geometry.y
    _, nearest_idx = tree.query([lon, lat])
    nearest_node = node_ids[nearest_idx]
    if 'population_density' in G.nodes[nearest_node]:
        G.nodes[nearest_node]['population_density'] += row['Z']
    else:
        G.nodes[nearest_node]['population_density'] = row['Z']

print("Population density added as node feature")

# ------------------------------
# 3. Load POIs and assign count per node
# ------------------------------
pois_gdf = gpd.read_file(POIS_GPKG)  # GeoPackage

for idx, row in pois_gdf.iterrows():
    lon, lat = row.geometry.x, row.geometry.y
    _, nearest_idx = tree.query([lon, lat])
    nearest_node = node_ids[nearest_idx]
    if 'poi_count' in G.nodes[nearest_node]:
        G.nodes[nearest_node]['poi_count'] += 1
    else:
        G.nodes[nearest_node]['poi_count'] = 1

print("POI count added as node feature")

# ------------------------------
# 4. Add transit routes as node features
# ------------------------------
transit_gdf = gpd.read_file(TRANSIT_GEOJSON)

for idx, route in transit_gdf.iterrows():
    coords = list(route.geometry.coords)
    for lon, lat in coords:
        _, nearest_idx = tree.query([lon, lat])
        nearest_node = node_ids[nearest_idx]
        if 'has_transit' in G.nodes[nearest_node]:
            G.nodes[nearest_node]['has_transit'] += 1
        else:
            G.nodes[nearest_node]['has_transit'] = 1

print("Transit coverage added as node feature")

# ------------------------------
# 5. Save enriched graph
# ------------------------------
nx.write_graphml(G, OUTPUT_GRAPHML)
print(f"Saved enriched graph with features to {OUTPUT_GRAPHML}")
print(f"Graph now has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")


Loaded road graph with 145131 nodes and 378090 edges
Population density added as node feature
POI count added as node feature
Transit coverage added as node feature
Saved enriched graph with features to ../data/lahore_full_graph.graphml
Graph now has 145131 nodes and 378090 edges


In [2]:
print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")

degrees = [d for n, d in G.degree()]
print(f"Average degree: {sum(degrees)/len(degrees):.2f}")


Number of nodes: 145131
Number of edges: 378090
Average degree: 5.21


In [6]:
# Ensure all edge lengths are numeric
for u, v, d in G.edges(data=True):
    if 'length' in d:
        try:
            d['length'] = float(d['length'])
        except ValueError:
            d['length'] = 1.0  # fallback if conversion fails
    else:
        d['length'] = 1.0  # default length if missing


In [ ]:
# Betweenness centrality (critical intersections)
bet_centrality = nx.betweenness_centrality(G, weight='length')  # length in meters if available
nx.set_node_attributes(G, bet_centrality, 'betweenness')

# Closeness centrality (accessibility measure)
closeness_centrality = nx.closeness_centrality(G, distance='length')
nx.set_node_attributes(G, closeness_centrality, 'closeness')

# Degree centrality (connectivity)
degree_centrality = nx.degree_centrality(G)
nx.set_node_attributes(G, degree_centrality, 'degree_centrality')


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x76bc282b8370>>
Traceback (most recent call last):
  File "/home/muhammadrehman/miniconda3/envs/sna/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 781, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [ ]:
nx.write_graphml(G, "../data/fullyFeatured.graphml")

In [4]:
from networkx.algorithms import community

communities_generator = community.greedy_modularity_communities(G)
node_community = {}
for i, comm in enumerate(communities_generator):
    for n in comm:
        node_community[n] = i

nx.set_node_attributes(G, node_community, 'community')
print(f"Detected {len(communities_generator)} communities")


Detected 208 communities


In [5]:
desert_threshold = 1000  # meters to nearest transit
for n, attr in G.nodes(data=True):
    if attr.get('dist_to_transit', float('inf')) > desert_threshold:
        G.nodes[n]['is_transit_desert'] = 1
    else:
        G.nodes[n]['is_transit_desert'] = 0


In [ ]:
if G.is_directed():
    print("The graph is directed")
else:
    print("The graph is undirected")